In [1]:
import os, sys
sys.path.append('lib')
import numpy as np
from InputDataProcessor import InputDataProcessor 

In [2]:
from distributeFunctions import distribute_602020
from GenericModel import GenericModel
from RecommenderModel import RecommenderModel
from distributeFunctions import distribute_602020
from dataFilters import csv_weather_data_prediction_filter, csv_weather_data_classification_filter
from tensorflow.keras.losses import BinaryCrossentropy,SparseCategoricalCrossentropy, MeanSquaredError
from tensorflow.keras.optimizers import Adam
from ModelConfig import ModelConfig

2024-05-17 05:51:43.941298: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-17 05:51:45.328287: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2024-05-17 05:51:45.328466: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [3]:
idp=InputDataProcessor(bucket_name='aiml_harveen', use_case='weather_elbasan/classification', file_name='elbasan2021.csv')


In [4]:
data_train,data_dev,data_test=idp.prepareCsvModelInputData(csv_weather_data_classification_filter,distribute_602020)

In [5]:
#os.environ["AIP_TRAINING_DATA_URI"] = "gs://aiml_harveen/c2w3_assignment/classification/training_data/data.csv"
#os.environ["AIP_VALIDATION_DATA_URI"] = "gs://aiml_harveen/c2w3_assignment/classification/dev_data/data.csv"
#os.environ["AIP_TEST_DATA_URI"] = "gs://aiml_harveen/c2w3_assignment/classification/test_data/data.csv"
#os.environ["AIP_MODEL_DIR"] = "gs://aiml_harveen/c2w3_assignment/classification/c2w3_model"

In [6]:
m,n=data_train.shape
data_train_x,data_train_y=np.split(data_train,[n-1],axis=1)

In [11]:
data_train_x.shape

(218, 6)

In [7]:
m,n=data_dev.shape
data_dev_x,data_dev_y=np.split(data_dev,[n-1],axis=1)

In [8]:
m,n=data_test.shape
data_test_x,data_test_y=np.split(data_test,[n-1],axis=1)

In [9]:
modelConfigList=[
    ModelConfig(
    model_type="multi-classification",
    layers=[["relu","relu","linear"]],
    regularizers=[[0.00,0.00,0.00]],
    neurons=[[120,40,6]],
    epochs=10,
    loss=SparseCategoricalCrossentropy(from_logits=True),
    optimizer=Adam(learning_rate=0.01),
    verbose=1,
    no_of_networks=1
    ),
    
#    ModelConfig(
#    model_type="classification",
#    layers=["relu","relu","relu","relu","sigmoid"],
#    regularizers=[0.02,0.02,0.02,0.02,0.02],
#    neurons=[32,32,16,16,1],
#    epochs=20,
#    loss=BinaryCrossentropy(),
#    optimizer=Adam(learning_rate=0.003))
]

2024-05-17 05:51:53.904324: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2024-05-17 05:51:53.904382: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-05-17 05:51:53.904413: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (instance-20240502-111332): /proc/driver/nvidia/version does not exist
2024-05-17 05:51:53.908744: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow 

In [10]:
gm=RecommenderModel(modelConfigList[0])
gm.train([data_train_x],data_train_y)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 6)]               0         
                                                                 
 sequential (Sequential)     (None, 6)                 5926      
                                                                 
 tf.math.l2_normalize (TFOpL  (None, 6)                0         
 ambda)                                                          
                                                                 
Total params: 5,926
Trainable params: 5,926
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
7/7 [==============================] - 1s 4ms/step - loss: 1.3247
Epoch 2/10
7/7 [==============================] - 0s 4ms/step - loss: 1.1576
Epoch 3/10
7/7 [==============================] - 0s 4ms/step - loss: 1.1356
Epoch 4/10
7/7 [==========

0.09174311926605505

In [28]:
gm.train_err

0.08

In [29]:
gm.validate()

10/10 [==============================] - 0s 2ms/step


In [30]:
gm.validation_err

0.0875

In [31]:
gm.test()

3/3 [==============================] - 0s 2ms/step


In [32]:
gm.test_err

0.175